In [1]:
!nvidia-smi

Mon Mar 14 05:38:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import cuml as cml
import cupy as pd

In [3]:
from numba import cuda
from cuml import ForestInference

import sklearn, sklearn.datasets, numpy as np

In [4]:
from sklearn import datasets

In [5]:
iris = datasets.load_iris()
from sklearn.metrics import accuracy_score
import xgboost as xgb

In [6]:
X = iris.data[:, :2]
y = iris.target

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from cuml.test.utils import array_equal

In [8]:
X_train, X_validation, y_train, y_validation = train_test_split(
    X, y, train_size=0.8,random_state=42)

In [9]:
def train_xgboost_model(X_train, y_train,
                        num_rounds, model_path):
    # set the xgboost model parameters
    params = { 'eval_metric':'error',
              
              'max_depth': 16}
    dtrain = xgb.DMatrix(X_train, label=y_train)
    # train the xgboost model
    bst = xgb.train(params, dtrain, num_rounds)

    # save the trained xgboost model
    bst.save_model(model_path)

    return bst

In [10]:
def predict_xgboost_model(X_validation, y_validation, xgb_model):

    # predict using the xgboost model
    dvalidation = xgb.DMatrix(X_validation, label=y_validation)
    xgb_preds = xgb_model.predict(dvalidation)

    # convert the predicted values from xgboost into class labels
    xgb_preds = np.around(xgb_preds)
    return xgb_preds

In [11]:
model_path = 'xgb.model'

# num of iterations for which the model is trained
num_rounds = 15

In [12]:
xgboost_model = train_xgboost_model(X_train, y_train,
                                    num_rounds, model_path)
fil_predsxgb = predict_xgboost_model(X_validation,y_validation,xgboost_model)
print(accuracy_score(fil_predsxgb,y_validation))

0.7


In [13]:
trained_model_preds = predict_xgboost_model(X_validation,
                                            y_validation,
                                            xgboost_model)

In [14]:
fm = ForestInference.load(filename=model_path,
                          algo='BATCH_TREE_REORG',
                          output_class=True,
                          threshold=0.50,
                          model_type='xgboost')

In [15]:
fil_preds = fm.predict(X_validation)
print(accuracy_score(fil_preds,y_validation))

0.6


In [16]:
print("The shape of predictions obtained from xgboost : ",(trained_model_preds).shape)
print("The shape of predictions obtained from FIL : ",(fil_preds).shape)
print("Are the predictions for xgboost and FIL the same : " ,   array_equal(trained_model_preds, fil_preds))

The shape of predictions obtained from xgboost :  (30,)
The shape of predictions obtained from FIL :  (30,)
Are the predictions for xgboost and FIL the same :  False
